In [ ]:
'''
This code:
1. Searches for app packages by searching on google play store
2. Search for open source links on F-droid based on the app-package name
'''

In [1]:
# !pip install google-play-scraper
# !pip install beautifulsoup4


In [6]:
from google_play_scraper import app
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm
import pandas as pd

In [7]:
def getHtmlContent(url):
    response=requests.get(url)
    htmlContent=BeautifulSoup(response.content,"html.parser")
    return htmlContent

In [4]:
#Get all apps from home assistant site
htmlContent = getHtmlContent("https://www.home-assistant.io/integrations/#all")
links = htmlContent.find_all("a")
ha_apps = []
for link in tqdm(links):
    href = str(link.get("href"))
    if "integrations" in href:
        if len(href.split("/"))>3:
            ha_apps.append(href.split("/")[-2])

100%|██████████████████████████████████████████████████████████████████████████████████████| 1303/1303 [00:00<?, ?it/s]


In [5]:
#Search with the app title, look for install button, if exists then grab the package name
#Create a dataframe consisting app title and package name
appPackageDF=pd.DataFrame([],columns=["App Title","Package"])
for app in tqdm(ha_apps):
    #print(app,end=" ==> ")
    searchURL="https://play.google.com/store/search?q="+app+"&c=apps"
    #print(searchURL)
    htmlContent = getHtmlContent(searchURL)
    #Looking for the install button, it is under itemprop="offers"
    allOffers = htmlContent.findAll("span",{"itemprop":"offers"})
    try:
        installComp=BeautifulSoup(allOffers[0].encode('utf-8'), "html.parser")
        meta=installComp.find("meta")
        package=meta["content"].split("=")[-2].split("&")[0]
        appPackageDF.loc[len(appPackageDF)]=[app,package]
        #print(package)
    except:
        #print("Error finding package for"+app)
        pass
appPackageDF.to_csv("scrapedPackages.csv")

100%|██████████████████████████████████████████████████████████████████████████████| 1207/1207 [10:38<00:00,  1.89it/s]


In [6]:
#Search package name on F-droid, if exists then grab the source code url
#Create a dataframe consisting app title, package name and source code url
appURL=pd.DataFrame([],columns=["App Title","Package","Source code URL"])
for appPackage in tqdm(appPackageDF["Package"].tolist()):
    appTitle=appPackageDF["App Title"][appPackageDF["Package"]==appPackage].tolist()[0]
    fdroidURL="https://f-droid.org/en/packages/"+appPackage
    htmlContent = getHtmlContent(fdroidURL)
    try:
        links = htmlContent.find_all('a', string='Source Code')
        href=links[0].get("href")
        appURL.loc[len(appURL)]=[appTitle,appPackage,href]
    except:
        pass
appURL.to_csv("scrapedPackages_SourceURL.csv")

100%|████████████████████████████████████████████████████████████████████████████████| 274/274 [01:55<00:00,  2.38it/s]


In [7]:
len(appURL)

14

In [40]:
# #Get all apps from home assistant site
# home="https://www.home-assistant.io"
# htmlContent = getHtmlContent("https://www.home-assistant.io/integrations/#all")
# links = htmlContent.find_all("a")

# for link in links:
#     href = str(link.get("href"))
#     if "integrations" in href:
#         if len(href.split("/"))>3:
#             appHtml=getHtmlContent(home+href)
#             tags= appHtml.find_all("a")
#             for tag in tags:
#                 text=tag.get_text()
#                 lookFor="on GitHub"
#                 if lookFor in text:
#                     integCode=tag.get("href")
#                     gitSourceHTML=getHtmlContent(integCode)
#                     gitSourceLinks=gitSourceHTML.find_all("a")
#                     for gsl in gitSourceLinks:
#                         try:
#                             if "test" in gsl.get("href").split("/")[-1].lower():
#                                 print(gsl.get("href"))
                                
#                         except:
#                             pass
                
                    
                
           
                

/login?return_to=https%3A%2F%2Fgithub.com%2Fhome-assistant%2Fcore%2Ftree%2Fdev%2Fhomeassistant%2Fcomponents%2Fmqtt_statestream
https://github.com/home-assistant/core/tree/{{ urlEncodedRefName }}/homeassistant/components/mqtt_statestream
https://github.com/home-assistant/core/tree/{{ urlEncodedRefName }}/homeassistant/components/mqtt_statestream
/home-assistant/core/commits/dev/homeassistant/components/mqtt_statestream
/home-assistant/core/tree/ea0e1c54d6eca0ffaf65e970cff3298d1b30e1c7/homeassistant/components/mqtt_statestream
/login?return_to=https%3A%2F%2Fgithub.com%2Fhome-assistant%2Fcore%2Ftree%2Fdev%2Fhomeassistant%2Fcomponents%2Fspeedtestdotnet
https://github.com/home-assistant/core/tree/{{ urlEncodedRefName }}/homeassistant/components/speedtestdotnet
https://github.com/home-assistant/core/tree/{{ urlEncodedRefName }}/homeassistant/components/speedtestdotnet
/home-assistant/core/commits/dev/homeassistant/components/speedtestdotnet
/home-assistant/core/tree/ea0e1c54d6eca0ffaf65e970c